# Labeling of Incidents

## Configuration file

In [1]:
configuration = 'predicting.yaml'

In [2]:
import os, sys
src_dir = os.environ['BASKERVILLE_ROOT'] + '/src'
if not os.path.lexists(src_dir):
    raise RuntimeError('Baskerville source dir does not exist!')
module_path = os.path.abspath(os.path.join(src_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

## Attack parameters

In [3]:
import time
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline
from baskerville.util.helpers import parse_config
from baskerville.util.baskerville_tools import BaskervilleDBTools
from baskerville.models.config import BaskervilleConfig
from baskerville.db import set_up_db
from baskerville.db.models import Attack, Attribute, RequestSet, Model  
from baskerville.models.engine import BaskervilleAnalyticsEngine

/usr/local/lib/python3.6/site-packages/sqlalchemy/sql/functions.py:67: SAWarning: The GenericFunction 'array_agg' is already registered and is going to be overridden.
  "is going to be overridden.".format(identifier)
/usr/local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
os.environ['TZ'] = 'UTC'
time.tzset()
conf_path = os.environ['BASKERVILLE_ROOT'] + f'/conf/{configuration}'
config = parse_config(path=conf_path)
config = BaskervilleConfig(config).validate()
features = config.engine.extra_features

config.database.maintenance = None
db_tools = BaskervilleDBTools(config.database)
db_tools.connect_to_db()

2021-01-19 00:34:12,024 baskerville.models.config.validate +208: DEBUG    [25912] Validating BaskervilleConfig...
2021-01-19 00:34:12,031 baskerville.models.config.validate +688: DEBUG    [25912] Validating DatabaseConfig...
2021-01-19 00:34:12,036 baskerville.models.config.validate +307: DEBUG    [25912] Validating EngineConfig...
2021-01-19 00:34:12,040 baskerville.models.config.validate +455: DEBUG    [25912] Validating SimulationConfig...
2021-01-19 00:34:12,048 baskerville.models.config.validate +966: DEBUG    [25912] Validating DataParsingConfig...
2021-01-19 00:34:12,438 baskerville.models.config.validate +938: DEBUG    [25912] Validating MetricsConfig...
2021-01-19 00:34:12,442 baskerville.models.config.validate +490: DEBUG    [25912] Validating TrainingConfig...
/root/baskerville/src/baskerville/models/config.py:340: UserWarning: Time bucket is NOT set to the default 120 seconds.
  warnings.warn('Time bucket is NOT set to the default 120 seconds.')
2021-01-19 00:34:12,450 bask

## Attack parameters

In [6]:
# target = 'nkd'
# table = 'request_sets_y2020_w14'
# fileter_column = 'stop'
# attack_start = '2020-04-03 09:49:00'
# attack_stop = '2020-04-03 10:49:00'
# regular_start = '2020-03-22 00:00:00'
# regular_stop = '2020-04-03 09:00:00'

# target = 'nordis'
# table = 'request_sets_y2020_w20'
# fileter_column = 'stop'
# attack_start = '2020-05-11 21:06:00'
# attack_stop = '2020-05-11 21:18:00'
# regular_start = '2020-05-11 00:00:00'
# regular_stop = '2020-05-11 21:00:00'

# target = 'balochcampaign'
# table = 'request_sets_y2020_w21'
# fileter_column = 'stop'
# attack_start = '2020-05-20 22:20:00'
# attack_stop = '2020-05-20 22:40:00'
# regular_start = '2020-05-20 18:00:00'
# regular_stop = '2020-05-20 22:00:00'

# target = 'stopblackface'
# table = 'request_sets_y2020_w24'
# fileter_column = 'stop'
# attack_start = '2020-06-08 14:00:00'
# attack_stop = '2020-06-08 14:08:00'
# regular_start = '2020-06-05 18:00:00'
# regular_stop = '2020-06-08 13:00:00'

# target = 'antifa'
# table = 'request_sets_y2020_w24'
# fileter_column = 'stop'
# attack_start = '2020-06-12 11:56:00'
# attack_stop = '2020-06-12 12:02:00'
# regular_start = '2020-06-12 00:00:00'
# regular_stop = '2020-06-12 11:52:00'

# target = 'losdanieles'
# fileter_column = 'stop'
# table = 'request_sets_y2020_w27'
# attack_start = '2020-07-05 14:28:00Z'
# attack_stop = '2020-07-05 14:46:00Z'
# regular_start = '2020-07-04 14:46:00Z'
# regular_stop = '2020-07-05 09:46:00Z'

# target = 'balochcampaign'
# table = 'request_sets'
# fileter_column = 'created_at'
# attack_start = '2020-07-09 08:34:00Z'
# attack_stop = '2020-07-09 09:56:00Z'
# regular_start = '2020-07-09 00:00:00Z'
# regular_stop = '2020-07-09 08:00:00Z'

# target = 'aletihadpress'
# table = 'request_sets'
# fileter_column = 'stop'
# attack_start = '2020-07-31 00:00:00Z'
# attack_stop = '2020-07-31 00:30:00Z'
# regular_start = '2020-07-31 02:02:00Z'
# regular_stop = '2020-07-31 10:00:00Z'

# id 18 - classifier
# id 19 - vladimir's list of ips
# target = 'informator'  
# table = 'request_sets'
# fileter_column = 'stop'
# attack_start = '2020-09-17 17:47:00Z'
# attack_stop = '2020-09-17 17:59:00Z'
# regular_start = '2020-09-17 16:00:00Z'
# regular_stop = '2020-09-17 17:00:00Z'

# target = 'antifa'  
# table = 'request_sets'
# fileter_column = 'stop'
# attack_start = '2020-10-05 19:46:00Z'
# attack_stop = '2020-10-05 20:18:00Z'
# regular_start = '2020-10-05 15:00:00Z'
# regular_stop = '2020-10-05 19:00:00Z'

# target = 'balochcampaign'  
# table = 'request_sets'
# fileter_column = 'stop'
# attack_start = '2020-10-14 10:20:00Z'
# attack_stop = '2020-10-14 10:24:00Z'
# regular_start = '2020-10-14 00:00:00Z'
# regular_stop = '2020-10-14 09:00:00Z'

# target = 'bunnyapps'  
# table = 'request_sets'
# fileter_column = 'stop'
# attack_start = '2020-10-22 08:48:00Z'
# attack_stop = '2020-10-22 09:02:00Z'
# regular_start = '2020-10-01 00:00:00Z'
# regular_stop = '2020-10-22 05:00:00Z'

# target = 'bunnyapps'  
# table = 'request_sets'
# fileter_column = 'stop'
# attack_start = '2020-10-22 09:37:00Z'
# attack_stop = '2020-10-22 09:41:00Z'
# regular_start = '2020-10-01 00:00:00Z'
# regular_stop = '2020-10-22 05:00:00Z'

table = 'request_sets'
fileter_column = 'stop'

# target = 'centre1'   24
# attack_start = '2021-01-14 14:36:00Z'
# attack_stop = '2021-01-14 15:36:00Z'
# regular_start = '2021-01-14 10:00:00Z'
# regular_stop = '2021-01-14 14:35:00Z'



# target = 'bankwatch'  not saved, not a clear attack
# attack_start = '2021-01-14 16:30:00Z'
# attack_stop = '2021-01-14 18:00:00Z'
# regular_start = '2021-01-14 00:00:00Z'
# regular_stop = '2021-01-14 16:25:00Z'

# target = 'alainet'  # id 25 'low and slow' manual list from Vladimir
# attack_start = '2021-01-17 16:44:00Z'
# attack_stop = '2021-01-17 18:00:00Z'
# regular_start = '2021-01-17 00:00:00Z'
# regular_stop = '2021-01-17 16:30:00Z'

target = 'baotiengdan'  # id 26
attack_start = '2021-01-26 08:35:00Z'
attack_stop = '2021-01-26 08:47:00Z'
regular_start = '2021-01-26 00:00:00Z'
regular_stop = '2021-01-26 08:30:00Z'

## Load Dataset

In [7]:
def load_dataset(query, db_tools):
    print('Querying database...')
    df = pd.read_sql(query, db_tools.engine)
    print(f'{len(df)} records retrieved.')
    num_ips = len(df['ip'].unique())
    print(f'{num_ips} unique IPs')
    print(f'Unwrapping features...')
    if len(df) > 0:
        df[[*df['features'][0].keys()]] = df[
            'features'
        ].apply(pd.Series)
        df.drop('features', axis=1, inplace=True)
        df.drop('host', axis=1, inplace=True)
        df.drop('country', axis=1, inplace=True)
        df.drop('request_total', axis=1, inplace=True)
        df.drop('minutes_total', axis=1, inplace=True)
    print(f'Unwrapping features complete.')
    df = df.fillna(0)
    return df

In [8]:
df_attack = load_dataset(f'select * from {table} where target=\'{target}\' and ' \
                                 f'{fileter_column} > \'{attack_start}\' and '\
                                 f'{fileter_column} < \'{attack_stop}\'', db_tools)

Querying database...
12136 records retrieved.
3343 unique IPs
Unwrapping features...
Unwrapping features complete.


In [9]:
df_regular = load_dataset(f'select * from {table} where target=\'{target}\' and ' \
                                 f'{fileter_column} > \'{regular_start}\' and '\
                                 f'{fileter_column} < \'{regular_stop}\'', db_tools)

Querying database...
22137 records retrieved.
5790 unique IPs
Unwrapping features...
Unwrapping features complete.


In [10]:
regular_ips = df_regular[['ip']].drop_duplicates()
df_attack=df_attack.merge(regular_ips[['ip']], on=['ip'], how='outer', indicator=True)
df_attack=df_attack[df_attack['_merge']=='left_only']

In [11]:
len(df_attack['ip'].unique())

3127

In [12]:
len(df_regular['ip'].unique())

5790

## Classifier

In [13]:
if 'host' in features:
    features.remove('host')
if 'country' in features:
    features.remove('country')
if 'host_country' in features:
    features.remove('host_country')
if 'request_total' in features:
    features.remove('request_total')
if 'minutes_total' in features:
    features.remove('minutes_total')

labels = np.ones(len(df_regular), dtype=int)
labels = np.append(labels, np.zeros(len(df_attack), dtype=int))
dataset = pd.concat([df_regular[features], df_attack[features]])

# scaler = StandardScaler()
# dataset = scaler.fit_transform(dataset[features].values)

model = GradientBoostingClassifier(
    n_estimators=500, random_state=777, 
    max_depth=12,
    max_features='auto',
    learning_rate = 0.05)
model.fit(dataset, labels)

#df_attack_scaled = scaler.transform(df_attack[features])
# predictions = model.predict(df_attack_scaled)

predictions = model.predict(df_attack[features])

In [14]:
incident = df_attack[['ip']].copy()
incident['predictions'] = predictions
attackers_predicted = incident[incident['predictions'] == 0][['ip']].drop_duplicates()
regulars_predicted = incident[incident['predictions'] == 1][['ip']].drop_duplicates()

incident_ips = incident[['ip']].drop_duplicates()
regular_ips = df_regular[['ip']].drop_duplicates()

attackers_vs_regular_traffic = pd.merge(regular_ips, attackers_predicted, how='inner', on=['ip'])
regulars_vs_incident = pd.merge(regular_ips, incident_ips, how='inner', on=['ip'])

print(f'Number of unique IPs in the incident = {len(incident_ips)}')
print(f'Number of predicted attackers = {len(attackers_predicted)}')
print(f'Number of predicted regulars = {len(incident_ips)-len(attackers_predicted)}')
print(f'Intersection predicted attackers in regular traffic = {len(attackers_vs_regular_traffic)}, {len(attackers_vs_regular_traffic)/len(regular_ips)*100:2.1f}%')
print(f'Intersection regular traffic in incident = {len(regulars_vs_incident)}')

# filter out the IPs from regular traffic
attackers = pd.merge(attackers_predicted, regulars_vs_incident, how='outer', on=['ip'], indicator=True)
attackers = attackers[attackers['_merge'] == 'left_only'][['ip']]

print(f'Final number of attackers IP: {len(attackers)}')

Number of unique IPs in the incident = 3127
Number of predicted attackers = 3118
Number of predicted regulars = 9
Intersection predicted attackers in regular traffic = 0, 0.0%
Intersection regular traffic in incident = 0
Final number of attackers IP: 3118


## Save Attack to DB

In [15]:
# attackers = df_attack[['ip']].drop_duplicates()

In [16]:
attack = Attack()
attack.start = attack_start
attack.stop = attack_stop
attack.target = target

attributes = []
for ip in attackers['ip']:
    a = Attribute()
    a.value = ip
    a.attacks.append(attack)
    attributes.append(a)

db_tools.session.add(attack)
db_tools.session.add_all(attributes)
db_tools.session.commit()
print(f'New attack saved in the database. id = {attack.id}')

New attack saved in the database. id = 26


## Save Attack (hardcoded list of IPs)

In [25]:
manual_ips = ['123.129.153.162', '119.179.216.72', '119.186.207.44', '123.129.153.73', '114.119.158.38', '119.179.217.193', '119.186.204.92', '119.186.207.16', '121.20.234.144', '101.66.50.227', '113.121.43.128', '119.116.190.138', '119.179.216.184', '119.186.204.192', '119.186.206.109', '119.186.207.184', '123.129.153.2', '123.129.154.227', '123.129.155.150', '175.171.176.34', '183.27.48.136', '222.191.175.50', '58.214.188.80', '61.52.76.59', '110.250.27.156', '116.2.67.59', '116.2.91.19', '119.179.216.137', '119.179.217.129', '119.179.217.161', '119.186.205.11', '119.186.205.134', '119.186.205.253', '123.129.153.129', '123.129.153.172', '123.153.62.245', '150.255.3.213', '183.27.49.9', '183.27.50.254', '39.88.94.134', '39.88.190.211', '39.184.77.245', '58.214.187.122', '58.214.187.176', '58.214.189.99', '58.214.190.186', '101.66.49.221', '101.66.91.174', '101.67.155.151', '110.250.25.79', '110.250.28.102', '114.119.130.43', '114.119.134.187', '114.119.147.166', '114.119.148.127', '114.119.155.36', '114.119.159.86', '115.202.75.230', '115.202.80.62', '116.2.64.10', '116.2.65.173', '116.2.69.141', '116.2.70.76', '116.2.74.104', '116.2.78.51', '116.2.80.140', '116.2.90.45', '116.2.90.77', '116.2.93.145', '117.85.13.74', '119.116.177.15', '119.116.182.124', '119.116.184.168', '119.116.186.82', '119.116.187.152', '119.116.189.44', '119.166.10.75', '119.179.216.233', '119.179.217.11', '119.186.206.4', '119.186.207.93', '119.186.207.103', '119.186.207.141', '119.186.207.166', '121.20.233.153', '121.20.234.205', '121.238.69.241', '123.129.153.12', '123.129.153.170', '123.129.154.113', '123.129.154.232', '123.129.155.84', '123.129.155.99', '123.153.60.182', '123.153.63.114', '123.169.34.43', '123.169.36.100', '124.90.105.22', '180.112.17.105', '183.27.48.65', '183.27.50.175', '183.188.94.122', '183.188.245.153', '27.210.128.5', '27.210.130.120', '27.219.144.60', '27.219.146.37', '27.219.147.108', '39.71.45.38', '39.88.188.205', '39.88.190.38', '39.88.191.213', '39.184.78.204', '42.91.10.152', '58.214.185.170', '58.214.187.218', '58.214.187.244', '58.214.190.55', '58.214.190.179', '58.214.191.189', '61.52.74.38', '61.52.74.97', '61.52.77.238', '61.52.78.65', '61.52.78.85', '61.52.79.10', '101.66.8.39', '101.66.10.114', '101.66.51.188', '101.66.51.227', '101.66.89.36', '101.67.154.90', '110.250.18.51', '110.250.20.190', '110.250.21.0', '110.250.26.113', '110.250.26.139', '110.250.29.65', '110.250.29.72', '110.250.29.175', '112.48.9.1', '112.48.9.43', '112.48.9.48', '112.48.9.111', '112.48.9.150', '112.48.9.223', '113.58.227.225', '113.58.231.29', '113.58.241.127', '113.58.244.179', '113.121.47.62', '113.234.160.35', '113.234.167.76', '113.234.168.45', '113.234.173.153', '114.119.142.217', '114.119.157.82', '114.119.158.112', '115.202.64.255', '115.202.68.37', '115.202.89.95', '115.202.91.147', '115.202.95.30', '115.202.95.182', '116.2.68.133', '116.2.69.179', '116.2.70.126', '116.2.72.186', '116.2.74.24', '116.2.74.135', '116.2.74.226', '116.2.76.63', '116.2.76.213', '116.2.82.98', '116.2.82.151', '116.2.84.11', '116.2.85.181', '116.2.87.104', '116.2.87.239', '116.2.89.102', '116.2.95.139', '116.2.95.203', '117.85.13.178', '117.85.15.167', '118.79.140.7', '118.79.142.181', '118.79.187.46', '119.116.176.124', '119.116.178.95', '119.116.178.163', '119.116.181.25', '119.116.184.28', '119.116.184.100', '119.116.186.5', '119.116.190.13', '119.116.190.202', '119.116.190.249', '119.116.191.244', '119.166.8.206', '119.166.9.121', '119.166.10.86', '119.166.15.103', '119.179.217.39', '121.20.232.133', '121.20.234.134', '121.20.235.128', '121.238.69.188', '121.238.69.195', '123.14.254.191', '123.129.152.179', '123.129.152.191', '123.129.153.102', '123.129.154.187', '123.129.155.65', '123.129.155.145', '123.153.61.76', '123.153.63.86', '123.153.63.239', '123.153.140.95', '123.153.142.124', '123.153.142.176', '124.90.105.235', '124.90.107.9', '124.90.107.28', '124.90.108.69', '124.90.108.114', '124.90.108.158', '124.90.109.0', '144.255.48.200', '150.255.2.6', '150.255.7.220', '175.162.112.185', '175.162.113.139', '175.171.176.26', '175.171.178.165', '175.171.178.210', '175.171.185.60', '180.104.195.229', '182.34.27.114', '182.34.27.171', '182.34.34.209', '182.121.67.161', '182.121.69.93', '182.121.69.255', '183.188.86.230', '183.188.92.59', '183.188.213.29', '183.188.242.52', '183.188.242.164', '183.188.245.233', '211.97.166.161', '211.97.166.179', '218.68.108.163', '221.197.66.89', '222.191.173.55', '27.210.241.222', '27.219.57.128', '39.82.250.216', '39.88.92.68', '39.88.188.91', '39.88.188.220', '39.88.189.77', '39.88.189.200', '39.88.191.100', '39.88.194.223', '39.88.195.163', '39.88.195.169', '39.88.195.187', '39.89.161.150', '39.89.162.195', '39.181.138.186', '39.181.142.222', '39.184.74.33', '39.184.74.208', '39.184.78.104', '42.91.10.22', '42.91.10.222', '42.91.12.167', '42.91.14.45', '42.91.14.229', '42.91.15.43', '42.91.15.95', '42.91.15.158', '42.91.15.192', '49.68.249.23', '49.68.251.89', '49.81.173.50', '49.81.173.166', '49.81.174.141', '49.81.175.96', '58.20.131.205', '58.20.131.222', '58.20.132.66', '58.214.184.69', '58.214.184.239', '58.214.185.230', '58.214.187.24', '58.214.187.145', '58.214.187.246', '58.214.189.98', '60.1.183.85', '60.24.14.1', '60.24.14.90', '60.24.14.183', '61.52.72.49', '61.52.72.74', '61.52.72.83', '61.52.72.111', '61.52.72.219', '61.52.73.14', '61.52.73.107', '61.52.74.0', '61.52.75.77', '61.52.76.74', '61.52.76.207', '61.52.77.90', '61.52.78.181', '61.52.78.239', '61.52.79.8', '61.240.226.52', '61.241.207.29', '61.241.207.116', '101.66.48.68', '101.66.50.65', '101.66.90.36', '101.66.90.236', '101.67.152.64', '101.67.153.54', '101.67.153.142', '101.67.153.156', '101.67.154.171', '101.67.155.81', '110.250.16.132', '110.250.18.178', '110.250.19.130', '110.250.20.58', '110.250.21.10', '110.250.23.26', '110.250.23.36', '110.250.28.171', '110.250.29.34', '110.250.29.140', '110.250.29.172', '112.48.9.41', '112.48.9.137', '112.48.9.175', '112.48.9.179', '112.48.9.183', '112.48.10.105', '112.248.61.18', '112.248.61.152', '113.58.226.238', '113.58.232.2', '113.58.233.118', '113.58.233.154', '113.58.237.203', '113.58.237.204', '113.58.238.190', '113.58.238.224', '113.58.239.158', '113.58.241.118', '113.58.242.192', '113.58.243.184', '113.58.244.161', '113.58.245.249', '113.58.246.47', '113.58.246.118', '113.58.247.247', '113.121.47.120', '113.124.84.23', '113.124.95.65', '113.124.95.247', '113.234.161.241', '113.234.165.191', '113.234.171.234', '114.119.135.152', '114.119.136.199', '114.119.155.232', '114.217.113.249', '114.234.152.58', '114.234.156.81', '115.202.65.221', '115.202.66.13', '115.202.69.81', '115.202.73.105', '115.202.74.199', '115.202.78.50', '115.202.79.129', '115.202.80.175', '115.202.80.205', '115.202.83.94', '115.202.84.129', '115.202.91.139', '115.202.94.56', '115.202.94.116', '115.202.95.167', '116.2.65.168', '116.2.70.77', '116.2.70.195', '116.2.72.135', '116.2.72.183', '116.2.72.253', '116.2.77.211', '116.2.79.55', '116.2.81.140', '116.2.81.146', '116.2.83.152', '116.2.83.229', '116.2.84.28', '116.2.84.167', '116.2.85.121', '116.2.86.43', '116.2.87.58', '116.2.87.151', '116.2.88.36', '116.2.88.69', '116.2.89.175', '116.2.89.239', '116.2.91.40', '116.2.91.45', '116.2.92.17', '116.2.94.113', '116.2.94.225', '116.2.95.131', '117.85.12.115', '117.85.13.152', '118.79.142.30', '118.79.142.104', '118.79.142.193', '118.79.187.205', '119.116.178.168', '119.116.180.41', '119.116.181.191', '119.116.182.90', '119.116.183.47', '119.116.184.201', '119.116.187.89', '119.116.187.136', '119.116.187.171', '119.116.188.117', '119.116.190.61', '119.116.190.186', '119.116.191.253', '119.166.15.111', '119.179.217.73', '119.186.206.95', '119.186.206.104', '119.186.206.134', '119.186.207.10', '119.186.207.94', '120.32.38.113', '121.20.233.11', '121.20.233.172', '121.20.233.233', '121.20.235.100', '121.20.235.161', '121.20.235.215', '121.20.236.51', '121.20.236.106', '121.20.237.78', '121.29.80.10', '121.238.68.191', '121.238.69.202', '123.14.249.86', '123.14.250.119', '123.14.255.72', '123.129.152.174', '123.129.153.37', '123.129.154.133', '123.129.155.47', '123.129.155.115', '123.153.61.118', '123.153.63.51', '123.153.114.55', '123.153.114.237', '123.153.140.104', '123.153.142.14', '123.169.35.222', '123.169.39.44', '123.169.39.217', '123.233.68.150', '123.233.69.9', '124.90.104.24', '124.90.104.46', '124.90.105.120', '124.90.107.110', '124.90.107.152', '124.90.107.158', '124.90.107.247', '125.104.226.102', '125.104.249.253', '125.123.66.122', '125.123.66.139', '144.255.49.38', '144.255.49.85', '150.255.0.243', '150.255.3.51', '150.255.3.158', '150.255.6.33', '171.116.79.133', '171.116.204.14', '171.116.207.114', '175.162.122.191', '175.162.122.232', '175.171.179.1', '180.104.239.122', '180.104.239.196', '180.104.239.214', '180.112.16.18', '180.112.16.230', '180.112.17.199', '180.112.19.191', '180.124.83.229', '182.34.16.38', '182.34.33.232', '182.34.37.233', '182.119.39.109', '182.121.68.220', '182.121.68.233', '182.121.70.229', '183.27.49.145', '183.27.49.174', '183.27.50.24', '183.27.50.136', '183.27.50.193', '183.27.50.209', '183.27.50.228', '183.27.51.138', '183.27.51.143', '183.27.51.148', '183.27.51.179', '183.27.51.184', '183.27.51.223', '183.188.92.47', '183.188.92.185', '183.188.93.51', '183.188.93.68', '183.188.94.33', '183.188.94.84', '183.188.95.23', '183.188.211.232', '183.188.211.252', '183.188.213.161', '183.188.242.35', '183.188.242.38', '183.188.244.105', '183.188.244.189', '183.250.16.184', '183.251.24.246', '218.68.102.238', '218.68.105.154', '218.68.107.13', '218.68.107.46', '218.68.108.130', '221.197.56.117', '221.197.66.193', '222.137.46.173', '222.137.47.41', '222.191.172.120', '222.191.173.41', '222.191.173.197', '222.191.174.87', '27.210.129.69', '27.210.129.88', '27.210.129.240', '27.210.130.31', '27.210.131.83', '27.219.57.236', '27.219.59.195', '27.219.144.43', '27.219.146.189', '27.219.147.123', '39.65.174.144', '39.71.45.5', '39.77.152.190', '39.77.153.66', '39.77.153.125', '39.77.154.212', '39.77.155.236', '39.82.38.245', '39.82.159.103', '39.82.168.190', '39.82.203.230', '39.82.248.4', '39.88.93.20', '39.88.93.137', '39.88.94.138', '39.88.94.139', '39.88.95.140', '39.88.191.129', '39.88.191.239', '39.88.193.219', '39.88.195.68', '39.89.156.121', '39.89.156.171', '39.89.160.61', '39.89.161.8', '39.91.109.171', '39.181.136.162', '39.181.137.8', '39.181.137.9', '39.181.138.34', '39.181.138.180', '39.181.138.218', '39.181.139.79', '39.181.139.155', '39.181.139.158', '39.181.141.24', '39.181.142.0', '39.181.142.43', '39.181.142.194', '39.181.142.242', '39.181.142.244', '39.181.143.208', '39.184.72.25', '39.184.72.89', '39.184.72.137', '39.184.72.205', '39.184.74.143', '39.184.74.243', '39.184.75.80', '39.184.75.129', '39.184.78.71', '39.184.78.216', '39.184.78.223', '42.48.196.111', '42.48.212.148', '42.48.212.226', '42.48.214.208', '42.91.8.132', '42.91.8.190', '42.91.9.4', '42.91.9.21', '42.91.9.168', '42.91.10.73', '42.91.10.136', '42.91.10.179', '42.91.10.198', '42.91.10.215', '42.91.10.244', '42.91.11.9', '42.91.11.101', '42.91.11.109', '42.91.11.116', '42.91.11.142', '42.91.12.18', '42.91.12.93', '42.91.12.149', '42.91.12.157', '42.91.12.194', '42.91.12.210', '42.91.13.96', '42.91.13.160', '42.91.13.213', '42.91.13.246', '42.91.14.6', '42.91.14.23', '42.91.14.72', '42.91.14.99', '42.91.14.178', '42.91.14.186', '42.91.14.252', '42.91.15.0', '42.91.15.34', '42.91.15.78', '42.91.15.86', '42.91.15.150', '42.91.15.151', '42.91.15.164', '42.91.15.216', '42.228.224.34', '42.228.224.117', '42.228.225.197', '42.235.151.161', '49.68.249.33', '49.68.250.110', '49.81.165.52', '49.81.172.114', '49.81.172.225', '49.81.173.65', '49.81.175.95', '49.81.175.244', '58.20.131.48', '58.20.131.53', '58.20.131.89', '58.20.131.102', '58.20.131.106', '58.20.131.112', '58.20.131.127', '58.20.131.132', '58.20.131.148', '58.20.131.151', '58.20.131.167', '58.20.131.178', '58.20.131.180', '58.20.131.188', '58.20.131.208', '58.20.131.238', '58.20.131.241', '58.20.131.248', '58.20.132.19', '58.20.132.88', '58.20.132.106', '58.20.132.123', '58.20.132.128', '58.20.132.138', '58.20.132.140', '58.20.132.171', '58.20.132.188', '58.20.132.201', '58.20.132.210', '58.20.132.212', '58.20.132.222', '58.20.132.235', '58.20.132.240', '58.214.185.88', '58.214.185.96', '58.214.185.102', '58.214.186.92', '58.214.186.248', '58.214.187.73', '58.214.188.246', '58.214.189.112', '58.214.189.158', '58.214.191.105', '58.214.191.165', '58.214.191.254', '59.52.176.11', '59.52.178.44', '59.52.178.168', '59.61.164.99', '59.61.164.120', '60.1.180.165', '60.1.181.137', '60.1.181.145', '60.1.182.29', '60.1.182.132', '60.1.182.157', '60.1.184.138', '60.1.185.43', '60.1.185.119', '60.1.188.105', '60.1.189.185', '60.1.193.230', '60.1.195.33', '60.1.195.120', '60.24.14.72', '60.24.14.85', '60.24.14.129', '60.24.14.145', '60.24.14.169', '60.24.14.208', '60.24.14.227', '61.52.72.53', '61.52.72.148', '61.52.72.171', '61.52.72.186', '61.52.73.93', '61.52.73.200', '61.52.75.157', '61.52.77.223', '61.52.78.129', '61.52.79.195', '61.52.82.87', '61.52.84.106', '61.241.206.29', '61.241.206.34', '61.241.206.57', '61.241.206.201', '61.241.207.4', '61.241.207.139', '61.241.207.153', '61.241.207.157', '61.241.207.170', '61.241.207.185', '61.241.207.220', '101.24.176.207', '101.24.178.95', '101.24.178.162', '101.24.184.19', '101.24.185.12', '101.24.188.110', '101.24.190.5', '101.24.191.3', '101.24.191.12', '101.24.191.79', '101.24.191.131', '101.66.48.77', '101.66.48.83', '101.66.51.198', '101.66.91.49', '101.67.152.65', '101.67.153.93', '101.67.153.167', '101.67.153.242', '101.67.154.172', '101.67.154.188', '101.67.155.8', '101.67.155.62', '101.67.155.167', '110.90.12.196', '110.250.16.44', '110.250.18.234', '110.250.19.169', '110.250.23.247', '110.250.24.70', '110.250.24.251', '110.250.25.98', '110.250.26.91', '110.250.27.44', '110.250.27.153', '110.250.27.172', '110.250.28.120', '110.250.29.25', '110.250.29.39', '110.250.29.73', '110.250.29.75', '110.250.29.82', '110.250.29.99', '110.250.29.127', '110.250.29.131', '111.164.179.49', '112.48.9.52', '112.48.9.59', '112.48.9.75', '112.48.9.80', '112.48.9.88', '112.48.9.89', '112.48.9.90', '112.48.9.91', '112.48.9.128', '112.48.9.130', '112.48.9.144', '112.48.9.154', '112.48.9.168', '112.48.9.184', '112.48.9.209', '112.48.9.242', '112.48.10.1', '112.48.10.6', '112.48.10.10', '112.48.10.27', '112.48.10.57', '112.48.10.63', '112.48.10.73', '112.48.10.74', '112.48.10.109', '112.48.10.111', '112.48.10.112', '112.248.60.71', '112.248.62.151', '112.255.28.231', '112.255.29.180', '113.58.224.10', '113.58.224.33', '113.58.224.115', '113.58.225.111', '113.58.226.117', '113.58.227.80', '113.58.227.210', '113.58.228.135', '113.58.229.202', '113.58.229.230', '113.58.230.167', '113.58.231.151', '113.58.232.224', '113.58.235.11', '113.58.235.15', '113.58.235.67', '113.58.236.113', '113.58.236.182', '113.58.237.10', '113.58.237.53', '113.58.238.7', '113.58.238.53', '113.58.238.129', '113.58.240.69', '113.58.240.176', '113.58.241.21', '113.58.242.219', '113.58.244.13', '113.58.244.58', '113.58.244.182', '113.58.244.226', '113.58.245.53', '113.58.245.183', '113.58.245.194', '113.58.247.148', '113.58.247.195', '113.58.247.220', '113.121.20.154', '113.121.23.117', '113.121.40.6', '113.121.40.28', '113.121.41.59', '113.121.44.27', '113.121.44.72', '113.121.46.4', '113.121.47.108', '113.124.85.63', '113.124.86.117', '113.124.86.197', '113.124.87.77', '113.124.94.98', '113.234.162.194', '113.234.163.31', '113.234.165.68', '113.234.165.71', '113.234.167.160', '113.234.168.127', '113.234.168.229', '113.234.170.83', '113.234.170.169', '113.234.173.64', '113.234.174.82', '113.234.174.147', '113.234.174.156', '113.234.174.190', '114.119.128.29', '114.119.128.56', '114.119.132.173', '114.119.134.182', '114.119.136.155', '114.119.137.43', '114.119.142.124', '114.119.145.163', '114.119.151.63', '114.119.161.177', '114.119.167.18', '114.217.113.218', '114.234.152.163', '114.234.152.164', '114.234.153.207', '114.234.155.68', '114.234.156.177', '114.234.157.239', '114.234.157.241', '114.234.158.8', '114.234.158.114', '114.234.158.130', '114.234.159.200', '114.234.159.209', '115.202.67.146', '115.202.67.191', '115.202.68.205', '115.202.69.171', '115.202.69.174', '115.202.70.251', '115.202.74.85', '115.202.74.213', '115.202.75.244', '115.202.76.151', '115.202.78.52', '115.202.83.20', '115.202.83.84', '115.202.83.105', '115.202.83.144', '115.202.86.51', '115.202.86.216', '115.202.87.62', '115.202.88.128', '115.202.89.207', '115.202.90.71', '115.202.90.187', '115.202.91.184', '115.202.94.35', '115.202.95.106', '115.202.95.235', '116.2.64.16', '116.2.64.172', '116.2.66.157', '116.2.66.173', '116.2.67.41', '116.2.67.97', '116.2.67.205', '116.2.68.65', '116.2.68.247', '116.2.70.242', '116.2.71.169', '116.2.73.7', '116.2.73.163', '116.2.74.195', '116.2.75.171', '116.2.80.17', '116.2.82.178', '116.2.82.182', '116.2.84.99', '116.2.85.73', '116.2.87.73', '116.2.87.89', '116.2.87.179', '116.2.88.143', '116.2.88.247', '116.2.91.168', '116.2.92.35', '116.2.93.33', '116.2.93.126', '116.2.93.173', '116.2.94.19', '116.2.94.155', '116.2.94.156', '116.2.95.18', '116.2.95.100', '116.2.95.152', '116.2.95.167', '116.2.95.173', '117.81.84.56', '118.79.140.180', '118.79.140.189', '118.79.143.51', '118.79.187.68', '118.79.187.234', '119.116.177.85', '119.116.178.50', '119.116.179.68', '119.116.180.57', '119.116.181.167', '119.116.182.4', '119.116.182.122', '119.116.182.193', '119.116.183.63', '119.116.188.175', '119.116.189.31', '119.116.189.99', '119.116.189.102', '119.116.189.113', '119.116.189.125', '119.116.189.144', '119.116.189.225', '119.116.190.44', '119.116.190.174', '119.116.191.84', '119.116.191.144', '119.116.191.156', '119.166.9.84', '119.166.9.242', '119.166.12.189', '119.166.13.140', '119.179.217.52', '119.179.217.203', '119.186.207.227', '120.32.38.16', '121.20.232.105', '121.20.232.140', '121.20.233.229', '121.20.234.76', '121.20.234.218', '121.20.234.228', '121.20.235.86', '121.20.236.172', '121.20.237.169', '121.29.81.224', '121.204.0.105', '121.238.68.34', '121.238.69.96', '123.14.248.35', '123.14.254.249', '123.129.153.14', '123.129.154.97', '123.153.60.143', '123.153.61.130', '123.153.61.163', '123.153.63.75', '123.153.112.82', '123.153.112.225', '123.153.114.186', '123.153.115.7', '123.153.115.220', '123.153.141.65', '123.169.34.241', '123.169.35.129', '123.169.35.175', '123.169.36.240', '123.169.37.234', '123.169.38.50', '123.169.39.175', '123.169.39.185', '123.232.126.96', '123.232.237.190', '123.232.238.0', '123.232.238.36', '123.232.238.47', '123.232.238.182', '123.232.239.19', '123.232.239.31', '123.232.239.54', '124.90.105.68', '124.90.105.104', '124.90.106.225', '124.90.107.78', '124.90.107.173', '124.90.107.241', '124.90.108.205', '124.90.109.117', '124.90.109.217', '124.133.40.169', '125.43.86.218', '125.77.0.93', '125.104.226.32', '125.104.229.143', '125.104.229.226', '125.104.230.233', '125.104.232.48', '125.104.232.166', '125.104.235.143', '125.104.240.152', '125.104.243.152', '125.104.243.238', '125.104.244.154', '125.104.249.105', '125.104.249.127', '125.104.250.106', '125.104.250.123', '125.123.66.117', '125.123.66.142', '125.123.67.89', '125.123.68.70', '125.123.68.96', '125.123.68.229', '125.123.69.47', '125.123.69.253', '125.123.70.38', '125.123.71.135', '125.123.71.211', '125.123.120.31', '125.123.120.242', '125.123.124.172', '125.123.124.211', '125.123.138.90', '125.123.142.176', '144.255.48.49', '144.255.48.239', '144.255.49.235', '150.255.0.164', '150.255.0.241', '150.255.1.110', '150.255.1.153', '150.255.1.181', '150.255.1.246', '150.255.2.147', '150.255.2.254', '150.255.3.34', '150.255.3.104', '150.255.3.203', '150.255.5.46', '171.116.76.175', '171.116.76.201', '171.116.76.245', '171.116.77.60', '171.116.77.116', '171.116.78.153', '171.116.79.16', '171.116.205.218', '171.116.206.138', '175.162.115.206', '175.162.116.154', '175.162.117.221', '175.162.123.121', '175.162.124.132', '175.171.179.101', '175.171.179.167', '180.104.239.90', '180.104.239.131', '180.104.239.183', '180.104.239.244', '180.112.16.198', '180.112.18.236', '180.112.19.252', '180.124.6.57', '180.124.50.241', '180.124.51.141', '182.34.16.111', '182.34.17.209', '182.34.18.0', '182.34.20.229', '182.34.22.85', '182.34.23.211', '182.34.26.181', '182.34.26.203', '182.34.26.244', '182.34.35.137', '182.34.35.180', '182.34.36.166', '182.101.48.84', '182.101.48.229', '182.101.48.230', '182.101.50.20', '182.101.50.87', '182.101.52.166', '182.101.52.229', '182.101.53.26', '182.101.54.76', '182.101.55.199', '182.101.58.246', '182.101.60.170', '182.101.61.96', '182.101.61.158', '182.101.61.163', '182.101.62.87', '182.119.39.138', '182.119.39.185', '182.121.64.187', '182.121.65.16', '182.121.65.175', '182.121.66.35', '182.121.67.185', '182.121.68.125', '182.121.71.135', '182.121.71.242', '183.27.48.135', '183.27.49.6', '183.27.49.123', '183.27.49.126', '183.27.49.131', '183.27.49.138', '183.27.49.200', '183.27.50.9', '183.27.50.28', '183.27.50.50', '183.27.50.113', '183.27.50.115', '183.27.50.140', '183.27.50.155', '183.27.50.166', '183.27.50.187', '183.27.50.202', '183.27.50.224', '183.27.50.227', '183.27.50.233', '183.27.51.81', '183.27.51.122', '183.27.51.142', '183.27.51.172', '183.27.51.183', '183.27.51.187', '183.141.35.147', '183.141.101.39', '183.185.84.26', '183.185.84.225', '183.185.85.154', '183.185.90.132', '183.185.90.193', '183.188.86.20', '183.188.86.25', '183.188.86.167', '183.188.91.68', '183.188.91.87', '183.188.91.97', '183.188.91.120', '183.188.91.121', '183.188.91.154', '183.188.91.185', '183.188.91.205', '183.188.92.107', '183.188.93.1', '183.188.93.53', '183.188.93.73', '183.188.93.76', '183.188.93.79', '183.188.93.182', '183.188.94.61', '183.188.94.72', '183.188.94.88', '183.188.94.168', '183.188.95.154', '183.188.95.249', '183.188.211.30', '183.188.211.215', '183.188.213.16', '183.188.242.132', '183.188.244.62', '183.188.244.223', '183.188.245.6', '183.188.245.11', '183.188.245.84', '183.188.245.101', '183.250.16.64', '183.250.16.97', '183.250.16.147', '183.250.16.163', '183.250.17.104', '183.250.18.8', '183.250.19.80', '183.250.19.144', '183.250.19.178', '183.250.19.190', '211.97.166.8', '211.97.166.12', '211.97.166.13', '211.97.166.57', '211.97.166.126', '211.97.166.156', '211.97.166.184', '211.97.166.201', '211.97.166.241', '211.97.167.2', '211.97.167.51', '211.97.167.100', '211.97.167.118', '211.97.167.162', '211.97.167.166', '211.97.167.183', '211.97.167.227', '211.97.167.231', '218.64.60.206', '218.68.102.21', '218.68.102.46', '218.68.102.75', '218.68.102.97', '218.68.105.34', '218.68.105.37', '218.68.105.92', '218.68.105.126', '218.68.105.143', '218.68.105.157', '218.68.105.210', '218.68.107.21', '218.68.107.25', '218.68.107.158', '218.68.107.159', '218.68.107.239', '218.73.118.32', '218.73.118.205', '220.160.18.27', '220.160.18.36', '220.175.36.21', '220.175.36.251', '220.175.37.62', '220.175.37.246', '220.175.54.38', '220.175.55.101', '220.175.55.126', '220.175.55.246', '220.175.70.212', '220.175.183.48', '220.175.183.55', '220.175.183.62', '220.175.183.78', '220.175.183.80', '220.177.9.41', '221.197.56.109', '221.197.66.139', '221.197.66.175', '221.197.66.185', '221.198.235.12', '221.198.235.60', '221.198.235.90', '221.198.235.140', '221.198.235.191', '222.76.6.25', '222.137.31.172', '222.191.172.243', '222.191.174.43', '222.191.174.196', '222.191.175.44', '222.191.175.86']

In [26]:
attack = Attack()
attack.start = attack_start
attack.stop = attack_stop
attack.target = target
attributes = []
for ip in manual_ips:
    a = Attribute()
    a.value = ip
    a.attacks.append(attack)
    attributes.append(a)
db_tools.session.add(attack)
db_tools.session.add_all(attributes)
db_tools.session.commit()

In [27]:
attack.id

25

In [24]:
banjax_ips = ['123.129.153.162', '119.179.216.72', '119.186.207.44', '123.129.153.73', '114.119.158.38', '119.179.217.193', '119.186.204.92', '119.186.207.16', '121.20.234.144', '101.66.50.227', '113.121.43.128', '119.116.190.138', '119.179.216.184', '119.186.204.192', '119.186.206.109', '119.186.207.184', '123.129.153.2', '123.129.154.227', '123.129.155.150', '175.171.176.34', '183.27.48.136', '222.191.175.50', '58.214.188.80', '61.52.76.59', '110.250.27.156', '116.2.67.59', '116.2.91.19', '119.179.216.137', '119.179.217.129', '119.179.217.161', '119.186.205.11', '119.186.205.134', '119.186.205.253', '123.129.153.129', '123.129.153.172', '123.153.62.245', '150.255.3.213', '183.27.49.9', '183.27.50.254', '39.88.94.134', '39.88.190.211', '39.184.77.245', '58.214.187.122', '58.214.187.176', '58.214.189.99', '58.214.190.186', '101.66.49.221', '101.66.91.174', '101.67.155.151', '110.250.25.79', '110.250.28.102', '114.119.130.43', '114.119.134.187', '114.119.147.166', '114.119.148.127', '114.119.155.36', '114.119.159.86', '115.202.75.230', '115.202.80.62', '116.2.64.10', '116.2.65.173', '116.2.69.141', '116.2.70.76', '116.2.74.104', '116.2.78.51', '116.2.80.140', '116.2.90.45', '116.2.90.77', '116.2.93.145', '117.85.13.74', '119.116.177.15', '119.116.182.124', '119.116.184.168', '119.116.186.82', '119.116.187.152', '119.116.189.44', '119.166.10.75', '119.179.216.233', '119.179.217.11', '119.186.206.4', '119.186.207.93', '119.186.207.103', '119.186.207.141', '119.186.207.166', '121.20.233.153', '121.20.234.205', '121.238.69.241', '123.129.153.12', '123.129.153.170', '123.129.154.113', '123.129.154.232', '123.129.155.84', '123.129.155.99', '123.153.60.182', '123.153.63.114', '123.169.34.43', '123.169.36.100', '124.90.105.22', '180.112.17.105', '183.27.48.65', '183.27.50.175', '183.188.94.122', '183.188.245.153', '27.210.128.5', '27.210.130.120', '27.219.144.60', '27.219.146.37', '27.219.147.108', '39.71.45.38', '39.88.188.205', '39.88.190.38', '39.88.191.213', '39.184.78.204', '42.91.10.152', '58.214.185.170', '58.214.187.218', '58.214.187.244', '58.214.190.55', '58.214.190.179', '58.214.191.189', '61.52.74.38', '61.52.74.97', '61.52.77.238', '61.52.78.65', '61.52.78.85', '61.52.79.10', '101.66.8.39', '101.66.10.114', '101.66.51.188', '101.66.51.227', '101.66.89.36', '101.67.154.90', '110.250.18.51', '110.250.20.190', '110.250.21.0', '110.250.26.113', '110.250.26.139', '110.250.29.65', '110.250.29.72', '110.250.29.175', '112.48.9.1', '112.48.9.43', '112.48.9.48', '112.48.9.111', '112.48.9.150', '112.48.9.223', '113.58.227.225', '113.58.231.29', '113.58.241.127', '113.58.244.179', '113.121.47.62', '113.234.160.35', '113.234.167.76', '113.234.168.45', '113.234.173.153', '114.119.142.217', '114.119.157.82', '114.119.158.112', '115.202.64.255', '115.202.68.37', '115.202.89.95', '115.202.91.147', '115.202.95.30', '115.202.95.182', '116.2.68.133', '116.2.69.179', '116.2.70.126', '116.2.72.186', '116.2.74.24', '116.2.74.135', '116.2.74.226', '116.2.76.63', '116.2.76.213', '116.2.82.98', '116.2.82.151', '116.2.84.11', '116.2.85.181', '116.2.87.104', '116.2.87.239', '116.2.89.102', '116.2.95.139', '116.2.95.203', '117.85.13.178', '117.85.15.167', '118.79.140.7', '118.79.142.181', '118.79.187.46', '119.116.176.124', '119.116.178.95', '119.116.178.163', '119.116.181.25', '119.116.184.28', '119.116.184.100', '119.116.186.5', '119.116.190.13', '119.116.190.202', '119.116.190.249', '119.116.191.244', '119.166.8.206', '119.166.9.121', '119.166.10.86', '119.166.15.103', '119.179.217.39', '121.20.232.133', '121.20.234.134', '121.20.235.128', '121.238.69.188', '121.238.69.195', '123.14.254.191', '123.129.152.179', '123.129.152.191', '123.129.153.102', '123.129.154.187', '123.129.155.65', '123.129.155.145', '123.153.61.76', '123.153.63.86', '123.153.63.239', '123.153.140.95', '123.153.142.124', '123.153.142.176', '124.90.105.235', '124.90.107.9', '124.90.107.28', '124.90.108.69', '124.90.108.114', '124.90.108.158', '124.90.109.0', '144.255.48.200', '150.255.2.6', '150.255.7.220', '175.162.112.185', '175.162.113.139', '175.171.176.26', '175.171.178.165', '175.171.178.210', '175.171.185.60', '180.104.195.229', '182.34.27.114', '182.34.27.171', '182.34.34.209', '182.121.67.161', '182.121.69.93', '182.121.69.255', '183.188.86.230', '183.188.92.59', '183.188.213.29', '183.188.242.52', '183.188.242.164', '183.188.245.233', '211.97.166.161', '211.97.166.179', '218.68.108.163', '221.197.66.89', '222.191.173.55', '27.210.241.222', '27.219.57.128', '39.82.250.216', '39.88.92.68', '39.88.188.91', '39.88.188.220', '39.88.189.77', '39.88.189.200', '39.88.191.100', '39.88.194.223', '39.88.195.163', '39.88.195.169', '39.88.195.187', '39.89.161.150', '39.89.162.195', '39.181.138.186', '39.181.142.222', '39.184.74.33', '39.184.74.208', '39.184.78.104', '42.91.10.22', '42.91.10.222', '42.91.12.167', '42.91.14.45', '42.91.14.229', '42.91.15.43', '42.91.15.95', '42.91.15.158', '42.91.15.192', '49.68.249.23', '49.68.251.89', '49.81.173.50', '49.81.173.166', '49.81.174.141', '49.81.175.96', '58.20.131.205', '58.20.131.222', '58.20.132.66', '58.214.184.69', '58.214.184.239', '58.214.185.230', '58.214.187.24', '58.214.187.145', '58.214.187.246', '58.214.189.98', '60.1.183.85', '60.24.14.1', '60.24.14.90', '60.24.14.183', '61.52.72.49', '61.52.72.74', '61.52.72.83', '61.52.72.111', '61.52.72.219', '61.52.73.14', '61.52.73.107', '61.52.74.0', '61.52.75.77', '61.52.76.74', '61.52.76.207', '61.52.77.90', '61.52.78.181', '61.52.78.239', '61.52.79.8', '61.240.226.52', '61.241.207.29', '61.241.207.116', '101.66.48.68', '101.66.50.65', '101.66.90.36', '101.66.90.236', '101.67.152.64', '101.67.153.54', '101.67.153.142', '101.67.153.156', '101.67.154.171', '101.67.155.81', '110.250.16.132', '110.250.18.178', '110.250.19.130', '110.250.20.58', '110.250.21.10', '110.250.23.26', '110.250.23.36', '110.250.28.171', '110.250.29.34', '110.250.29.140', '110.250.29.172', '112.48.9.41', '112.48.9.137', '112.48.9.175', '112.48.9.179', '112.48.9.183', '112.48.10.105', '112.248.61.18', '112.248.61.152', '113.58.226.238', '113.58.232.2', '113.58.233.118', '113.58.233.154', '113.58.237.203', '113.58.237.204', '113.58.238.190', '113.58.238.224', '113.58.239.158', '113.58.241.118', '113.58.242.192', '113.58.243.184', '113.58.244.161', '113.58.245.249', '113.58.246.47', '113.58.246.118', '113.58.247.247', '113.121.47.120', '113.124.84.23', '113.124.95.65', '113.124.95.247', '113.234.161.241', '113.234.165.191', '113.234.171.234', '114.119.135.152', '114.119.136.199', '114.119.155.232', '114.217.113.249', '114.234.152.58', '114.234.156.81', '115.202.65.221', '115.202.66.13', '115.202.69.81', '115.202.73.105', '115.202.74.199', '115.202.78.50', '115.202.79.129', '115.202.80.175', '115.202.80.205', '115.202.83.94', '115.202.84.129', '115.202.91.139', '115.202.94.56', '115.202.94.116', '115.202.95.167', '116.2.65.168', '116.2.70.77', '116.2.70.195', '116.2.72.135', '116.2.72.183', '116.2.72.253', '116.2.77.211', '116.2.79.55', '116.2.81.140', '116.2.81.146', '116.2.83.152', '116.2.83.229', '116.2.84.28', '116.2.84.167', '116.2.85.121', '116.2.86.43', '116.2.87.58', '116.2.87.151', '116.2.88.36', '116.2.88.69', '116.2.89.175', '116.2.89.239', '116.2.91.40', '116.2.91.45', '116.2.92.17', '116.2.94.113', '116.2.94.225', '116.2.95.131', '117.85.12.115', '117.85.13.152', '118.79.142.30', '118.79.142.104', '118.79.142.193', '118.79.187.205', '119.116.178.168', '119.116.180.41', '119.116.181.191', '119.116.182.90', '119.116.183.47', '119.116.184.201', '119.116.187.89', '119.116.187.136', '119.116.187.171', '119.116.188.117', '119.116.190.61', '119.116.190.186', '119.116.191.253', '119.166.15.111', '119.179.217.73', '119.186.206.95', '119.186.206.104', '119.186.206.134', '119.186.207.10', '119.186.207.94', '120.32.38.113', '121.20.233.11', '121.20.233.172', '121.20.233.233', '121.20.235.100', '121.20.235.161', '121.20.235.215', '121.20.236.51', '121.20.236.106', '121.20.237.78', '121.29.80.10', '121.238.68.191', '121.238.69.202', '123.14.249.86', '123.14.250.119', '123.14.255.72', '123.129.152.174', '123.129.153.37', '123.129.154.133', '123.129.155.47', '123.129.155.115', '123.153.61.118', '123.153.63.51', '123.153.114.55', '123.153.114.237', '123.153.140.104', '123.153.142.14', '123.169.35.222', '123.169.39.44', '123.169.39.217', '123.233.68.150', '123.233.69.9', '124.90.104.24', '124.90.104.46', '124.90.105.120', '124.90.107.110', '124.90.107.152', '124.90.107.158', '124.90.107.247', '125.104.226.102', '125.104.249.253', '125.123.66.122', '125.123.66.139', '144.255.49.38', '144.255.49.85', '150.255.0.243', '150.255.3.51', '150.255.3.158', '150.255.6.33', '171.116.79.133', '171.116.204.14', '171.116.207.114', '175.162.122.191', '175.162.122.232', '175.171.179.1', '180.104.239.122', '180.104.239.196', '180.104.239.214', '180.112.16.18', '180.112.16.230', '180.112.17.199', '180.112.19.191', '180.124.83.229', '182.34.16.38', '182.34.33.232', '182.34.37.233', '182.119.39.109', '182.121.68.220', '182.121.68.233', '182.121.70.229', '183.27.49.145', '183.27.49.174', '183.27.50.24', '183.27.50.136', '183.27.50.193', '183.27.50.209', '183.27.50.228', '183.27.51.138', '183.27.51.143', '183.27.51.148', '183.27.51.179', '183.27.51.184', '183.27.51.223', '183.188.92.47', '183.188.92.185', '183.188.93.51', '183.188.93.68', '183.188.94.33', '183.188.94.84', '183.188.95.23', '183.188.211.232', '183.188.211.252', '183.188.213.161', '183.188.242.35', '183.188.242.38', '183.188.244.105', '183.188.244.189', '183.250.16.184', '183.251.24.246', '218.68.102.238', '218.68.105.154', '218.68.107.13', '218.68.107.46', '218.68.108.130', '221.197.56.117', '221.197.66.193', '222.137.46.173', '222.137.47.41', '222.191.172.120', '222.191.173.41', '222.191.173.197', '222.191.174.87', '27.210.129.69', '27.210.129.88', '27.210.129.240', '27.210.130.31', '27.210.131.83', '27.219.57.236', '27.219.59.195', '27.219.144.43', '27.219.146.189', '27.219.147.123', '39.65.174.144', '39.71.45.5', '39.77.152.190', '39.77.153.66', '39.77.153.125', '39.77.154.212', '39.77.155.236', '39.82.38.245', '39.82.159.103', '39.82.168.190', '39.82.203.230', '39.82.248.4', '39.88.93.20', '39.88.93.137', '39.88.94.138', '39.88.94.139', '39.88.95.140', '39.88.191.129', '39.88.191.239', '39.88.193.219', '39.88.195.68', '39.89.156.121', '39.89.156.171', '39.89.160.61', '39.89.161.8', '39.91.109.171', '39.181.136.162', '39.181.137.8', '39.181.137.9', '39.181.138.34', '39.181.138.180', '39.181.138.218', '39.181.139.79', '39.181.139.155', '39.181.139.158', '39.181.141.24', '39.181.142.0', '39.181.142.43', '39.181.142.194', '39.181.142.242', '39.181.142.244', '39.181.143.208', '39.184.72.25', '39.184.72.89', '39.184.72.137', '39.184.72.205', '39.184.74.143', '39.184.74.243', '39.184.75.80', '39.184.75.129', '39.184.78.71', '39.184.78.216', '39.184.78.223', '42.48.196.111', '42.48.212.148', '42.48.212.226', '42.48.214.208', '42.91.8.132', '42.91.8.190', '42.91.9.4', '42.91.9.21', '42.91.9.168', '42.91.10.73', '42.91.10.136', '42.91.10.179', '42.91.10.198', '42.91.10.215', '42.91.10.244', '42.91.11.9', '42.91.11.101', '42.91.11.109', '42.91.11.116', '42.91.11.142', '42.91.12.18', '42.91.12.93', '42.91.12.149', '42.91.12.157', '42.91.12.194', '42.91.12.210', '42.91.13.96', '42.91.13.160', '42.91.13.213', '42.91.13.246', '42.91.14.6', '42.91.14.23', '42.91.14.72', '42.91.14.99', '42.91.14.178', '42.91.14.186', '42.91.14.252', '42.91.15.0', '42.91.15.34', '42.91.15.78', '42.91.15.86', '42.91.15.150', '42.91.15.151', '42.91.15.164', '42.91.15.216', '42.228.224.34', '42.228.224.117', '42.228.225.197', '42.235.151.161', '49.68.249.33', '49.68.250.110', '49.81.165.52', '49.81.172.114', '49.81.172.225', '49.81.173.65', '49.81.175.95', '49.81.175.244', '58.20.131.48', '58.20.131.53', '58.20.131.89', '58.20.131.102', '58.20.131.106', '58.20.131.112', '58.20.131.127', '58.20.131.132', '58.20.131.148', '58.20.131.151', '58.20.131.167', '58.20.131.178', '58.20.131.180', '58.20.131.188', '58.20.131.208', '58.20.131.238', '58.20.131.241', '58.20.131.248', '58.20.132.19', '58.20.132.88', '58.20.132.106', '58.20.132.123', '58.20.132.128', '58.20.132.138', '58.20.132.140', '58.20.132.171', '58.20.132.188', '58.20.132.201', '58.20.132.210', '58.20.132.212', '58.20.132.222', '58.20.132.235', '58.20.132.240', '58.214.185.88', '58.214.185.96', '58.214.185.102', '58.214.186.92', '58.214.186.248', '58.214.187.73', '58.214.188.246', '58.214.189.112', '58.214.189.158', '58.214.191.105', '58.214.191.165', '58.214.191.254', '59.52.176.11', '59.52.178.44', '59.52.178.168', '59.61.164.99', '59.61.164.120', '60.1.180.165', '60.1.181.137', '60.1.181.145', '60.1.182.29', '60.1.182.132', '60.1.182.157', '60.1.184.138', '60.1.185.43', '60.1.185.119', '60.1.188.105', '60.1.189.185', '60.1.193.230', '60.1.195.33', '60.1.195.120', '60.24.14.72', '60.24.14.85', '60.24.14.129', '60.24.14.145', '60.24.14.169', '60.24.14.208', '60.24.14.227', '61.52.72.53', '61.52.72.148', '61.52.72.171', '61.52.72.186', '61.52.73.93', '61.52.73.200', '61.52.75.157', '61.52.77.223', '61.52.78.129', '61.52.79.195', '61.52.82.87', '61.52.84.106', '61.241.206.29', '61.241.206.34', '61.241.206.57', '61.241.206.201', '61.241.207.4', '61.241.207.139', '61.241.207.153', '61.241.207.157', '61.241.207.170', '61.241.207.185', '61.241.207.220', '101.24.176.207', '101.24.178.95', '101.24.178.162', '101.24.184.19', '101.24.185.12', '101.24.188.110', '101.24.190.5', '101.24.191.3', '101.24.191.12', '101.24.191.79', '101.24.191.131', '101.66.48.77', '101.66.48.83', '101.66.51.198', '101.66.91.49', '101.67.152.65', '101.67.153.93', '101.67.153.167', '101.67.153.242', '101.67.154.172', '101.67.154.188', '101.67.155.8', '101.67.155.62', '101.67.155.167', '110.90.12.196', '110.250.16.44', '110.250.18.234', '110.250.19.169', '110.250.23.247', '110.250.24.70', '110.250.24.251', '110.250.25.98', '110.250.26.91', '110.250.27.44', '110.250.27.153', '110.250.27.172', '110.250.28.120', '110.250.29.25', '110.250.29.39', '110.250.29.73', '110.250.29.75', '110.250.29.82', '110.250.29.99', '110.250.29.127', '110.250.29.131', '111.164.179.49', '112.48.9.52', '112.48.9.59', '112.48.9.75', '112.48.9.80', '112.48.9.88', '112.48.9.89', '112.48.9.90', '112.48.9.91', '112.48.9.128', '112.48.9.130', '112.48.9.144', '112.48.9.154', '112.48.9.168', '112.48.9.184', '112.48.9.209', '112.48.9.242', '112.48.10.1', '112.48.10.6', '112.48.10.10', '112.48.10.27', '112.48.10.57', '112.48.10.63', '112.48.10.73', '112.48.10.74', '112.48.10.109', '112.48.10.111', '112.48.10.112', '112.248.60.71', '112.248.62.151', '112.255.28.231', '112.255.29.180', '113.58.224.10', '113.58.224.33', '113.58.224.115', '113.58.225.111', '113.58.226.117', '113.58.227.80', '113.58.227.210', '113.58.228.135', '113.58.229.202', '113.58.229.230', '113.58.230.167', '113.58.231.151', '113.58.232.224', '113.58.235.11', '113.58.235.15', '113.58.235.67', '113.58.236.113', '113.58.236.182', '113.58.237.10', '113.58.237.53', '113.58.238.7', '113.58.238.53', '113.58.238.129', '113.58.240.69', '113.58.240.176', '113.58.241.21', '113.58.242.219', '113.58.244.13', '113.58.244.58', '113.58.244.182', '113.58.244.226', '113.58.245.53', '113.58.245.183', '113.58.245.194', '113.58.247.148', '113.58.247.195', '113.58.247.220', '113.121.20.154', '113.121.23.117', '113.121.40.6', '113.121.40.28', '113.121.41.59', '113.121.44.27', '113.121.44.72', '113.121.46.4', '113.121.47.108', '113.124.85.63', '113.124.86.117', '113.124.86.197', '113.124.87.77', '113.124.94.98', '113.234.162.194', '113.234.163.31', '113.234.165.68', '113.234.165.71', '113.234.167.160', '113.234.168.127', '113.234.168.229', '113.234.170.83', '113.234.170.169', '113.234.173.64', '113.234.174.82', '113.234.174.147', '113.234.174.156', '113.234.174.190', '114.119.128.29', '114.119.128.56', '114.119.132.173', '114.119.134.182', '114.119.136.155', '114.119.137.43', '114.119.142.124', '114.119.145.163', '114.119.151.63', '114.119.161.177', '114.119.167.18', '114.217.113.218', '114.234.152.163', '114.234.152.164', '114.234.153.207', '114.234.155.68', '114.234.156.177', '114.234.157.239', '114.234.157.241', '114.234.158.8', '114.234.158.114', '114.234.158.130', '114.234.159.200', '114.234.159.209', '115.202.67.146', '115.202.67.191', '115.202.68.205', '115.202.69.171', '115.202.69.174', '115.202.70.251', '115.202.74.85', '115.202.74.213', '115.202.75.244', '115.202.76.151', '115.202.78.52', '115.202.83.20', '115.202.83.84', '115.202.83.105', '115.202.83.144', '115.202.86.51', '115.202.86.216', '115.202.87.62', '115.202.88.128', '115.202.89.207', '115.202.90.71', '115.202.90.187', '115.202.91.184', '115.202.94.35', '115.202.95.106', '115.202.95.235', '116.2.64.16', '116.2.64.172', '116.2.66.157', '116.2.66.173', '116.2.67.41', '116.2.67.97', '116.2.67.205', '116.2.68.65', '116.2.68.247', '116.2.70.242', '116.2.71.169', '116.2.73.7', '116.2.73.163', '116.2.74.195', '116.2.75.171', '116.2.80.17', '116.2.82.178', '116.2.82.182', '116.2.84.99', '116.2.85.73', '116.2.87.73', '116.2.87.89', '116.2.87.179', '116.2.88.143', '116.2.88.247', '116.2.91.168', '116.2.92.35', '116.2.93.33', '116.2.93.126', '116.2.93.173', '116.2.94.19', '116.2.94.155', '116.2.94.156', '116.2.95.18', '116.2.95.100', '116.2.95.152', '116.2.95.167', '116.2.95.173', '117.81.84.56', '118.79.140.180', '118.79.140.189', '118.79.143.51', '118.79.187.68', '118.79.187.234', '119.116.177.85', '119.116.178.50', '119.116.179.68', '119.116.180.57', '119.116.181.167', '119.116.182.4', '119.116.182.122', '119.116.182.193', '119.116.183.63', '119.116.188.175', '119.116.189.31', '119.116.189.99', '119.116.189.102', '119.116.189.113', '119.116.189.125', '119.116.189.144', '119.116.189.225', '119.116.190.44', '119.116.190.174', '119.116.191.84', '119.116.191.144', '119.116.191.156', '119.166.9.84', '119.166.9.242', '119.166.12.189', '119.166.13.140', '119.179.217.52', '119.179.217.203', '119.186.207.227', '120.32.38.16', '121.20.232.105', '121.20.232.140', '121.20.233.229', '121.20.234.76', '121.20.234.218', '121.20.234.228', '121.20.235.86', '121.20.236.172', '121.20.237.169', '121.29.81.224', '121.204.0.105', '121.238.68.34', '121.238.69.96', '123.14.248.35', '123.14.254.249', '123.129.153.14', '123.129.154.97', '123.153.60.143', '123.153.61.130', '123.153.61.163', '123.153.63.75', '123.153.112.82', '123.153.112.225', '123.153.114.186', '123.153.115.7', '123.153.115.220', '123.153.141.65', '123.169.34.241', '123.169.35.129', '123.169.35.175', '123.169.36.240', '123.169.37.234', '123.169.38.50', '123.169.39.175', '123.169.39.185', '123.232.126.96', '123.232.237.190', '123.232.238.0', '123.232.238.36', '123.232.238.47', '123.232.238.182', '123.232.239.19', '123.232.239.31', '123.232.239.54', '124.90.105.68', '124.90.105.104', '124.90.106.225', '124.90.107.78', '124.90.107.173', '124.90.107.241', '124.90.108.205', '124.90.109.117', '124.90.109.217', '124.133.40.169', '125.43.86.218', '125.77.0.93', '125.104.226.32', '125.104.229.143', '125.104.229.226', '125.104.230.233', '125.104.232.48', '125.104.232.166', '125.104.235.143', '125.104.240.152', '125.104.243.152', '125.104.243.238', '125.104.244.154', '125.104.249.105', '125.104.249.127', '125.104.250.106', '125.104.250.123', '125.123.66.117', '125.123.66.142', '125.123.67.89', '125.123.68.70', '125.123.68.96', '125.123.68.229', '125.123.69.47', '125.123.69.253', '125.123.70.38', '125.123.71.135', '125.123.71.211', '125.123.120.31', '125.123.120.242', '125.123.124.172', '125.123.124.211', '125.123.138.90', '125.123.142.176', '144.255.48.49', '144.255.48.239', '144.255.49.235', '150.255.0.164', '150.255.0.241', '150.255.1.110', '150.255.1.153', '150.255.1.181', '150.255.1.246', '150.255.2.147', '150.255.2.254', '150.255.3.34', '150.255.3.104', '150.255.3.203', '150.255.5.46', '171.116.76.175', '171.116.76.201', '171.116.76.245', '171.116.77.60', '171.116.77.116', '171.116.78.153', '171.116.79.16', '171.116.205.218', '171.116.206.138', '175.162.115.206', '175.162.116.154', '175.162.117.221', '175.162.123.121', '175.162.124.132', '175.171.179.101', '175.171.179.167', '180.104.239.90', '180.104.239.131', '180.104.239.183', '180.104.239.244', '180.112.16.198', '180.112.18.236', '180.112.19.252', '180.124.6.57', '180.124.50.241', '180.124.51.141', '182.34.16.111', '182.34.17.209', '182.34.18.0', '182.34.20.229', '182.34.22.85', '182.34.23.211', '182.34.26.181', '182.34.26.203', '182.34.26.244', '182.34.35.137', '182.34.35.180', '182.34.36.166', '182.101.48.84', '182.101.48.229', '182.101.48.230', '182.101.50.20', '182.101.50.87', '182.101.52.166', '182.101.52.229', '182.101.53.26', '182.101.54.76', '182.101.55.199', '182.101.58.246', '182.101.60.170', '182.101.61.96', '182.101.61.158', '182.101.61.163', '182.101.62.87', '182.119.39.138', '182.119.39.185', '182.121.64.187', '182.121.65.16', '182.121.65.175', '182.121.66.35', '182.121.67.185', '182.121.68.125', '182.121.71.135', '182.121.71.242', '183.27.48.135', '183.27.49.6', '183.27.49.123', '183.27.49.126', '183.27.49.131', '183.27.49.138', '183.27.49.200', '183.27.50.9', '183.27.50.28', '183.27.50.50', '183.27.50.113', '183.27.50.115', '183.27.50.140', '183.27.50.155', '183.27.50.166', '183.27.50.187', '183.27.50.202', '183.27.50.224', '183.27.50.227', '183.27.50.233', '183.27.51.81', '183.27.51.122', '183.27.51.142', '183.27.51.172', '183.27.51.183', '183.27.51.187', '183.141.35.147', '183.141.101.39', '183.185.84.26', '183.185.84.225', '183.185.85.154', '183.185.90.132', '183.185.90.193', '183.188.86.20', '183.188.86.25', '183.188.86.167', '183.188.91.68', '183.188.91.87', '183.188.91.97', '183.188.91.120', '183.188.91.121', '183.188.91.154', '183.188.91.185', '183.188.91.205', '183.188.92.107', '183.188.93.1', '183.188.93.53', '183.188.93.73', '183.188.93.76', '183.188.93.79', '183.188.93.182', '183.188.94.61', '183.188.94.72', '183.188.94.88', '183.188.94.168', '183.188.95.154', '183.188.95.249', '183.188.211.30', '183.188.211.215', '183.188.213.16', '183.188.242.132', '183.188.244.62', '183.188.244.223', '183.188.245.6', '183.188.245.11', '183.188.245.84', '183.188.245.101', '183.250.16.64', '183.250.16.97', '183.250.16.147', '183.250.16.163', '183.250.17.104', '183.250.18.8', '183.250.19.80', '183.250.19.144', '183.250.19.178', '183.250.19.190', '211.97.166.8', '211.97.166.12', '211.97.166.13', '211.97.166.57', '211.97.166.126', '211.97.166.156', '211.97.166.184', '211.97.166.201', '211.97.166.241', '211.97.167.2', '211.97.167.51', '211.97.167.100', '211.97.167.118', '211.97.167.162', '211.97.167.166', '211.97.167.183', '211.97.167.227', '211.97.167.231', '218.64.60.206', '218.68.102.21', '218.68.102.46', '218.68.102.75', '218.68.102.97', '218.68.105.34', '218.68.105.37', '218.68.105.92', '218.68.105.126', '218.68.105.143', '218.68.105.157', '218.68.105.210', '218.68.107.21', '218.68.107.25', '218.68.107.158', '218.68.107.159', '218.68.107.239', '218.73.118.32', '218.73.118.205', '220.160.18.27', '220.160.18.36', '220.175.36.21', '220.175.36.251', '220.175.37.62', '220.175.37.246', '220.175.54.38', '220.175.55.101', '220.175.55.126', '220.175.55.246', '220.175.70.212', '220.175.183.48', '220.175.183.55', '220.175.183.62', '220.175.183.78', '220.175.183.80', '220.177.9.41', '221.197.56.109', '221.197.66.139', '221.197.66.175', '221.197.66.185', '221.198.235.12', '221.198.235.60', '221.198.235.90', '221.198.235.140', '221.198.235.191', '222.76.6.25', '222.137.31.172', '222.191.172.243', '222.191.174.43', '222.191.174.196', '222.191.175.44', '222.191.175.86']

In [53]:
len(banjax_ips)

301

In [54]:
attack_ips = attackers['ip'].values.tolist()

In [55]:
len(attack_ips)

7584

In [56]:
len(banjax_ips and attack_ips)

7584

In [59]:
len(list(set(banjax_ips) & set(attack_ips)))



296

In [61]:
for v in attack_ips:
    print(v)

1.186.151.206
102.68.134.42
102.68.134.46
103.114.10.250
103.122.35.229
103.142.68.154
103.16.133.11
103.212.92.254
103.216.48.83
116.212.109.195
121.52.71.23
124.41.211.196
139.180.135.7
139.255.89.242
147.78.161.158
150.129.54.111
152.204.132.123
109.41.66.89
110.34.3.145
173.82.62.19
181.53.37.214
181.54.109.250
181.54.54.156
181.55.152.8
181.58.3.97
181.61.205.95
181.62.245.247
181.62.52.43
185.88.173.47
186.102.36.230
186.146.204.68
186.154.190.215
174.48.31.144
176.197.145.162
186.233.104.164
186.235.157.51
186.28.194.183
186.30.73.20
186.31.177.226
190.24.122.189
190.24.122.80
190.24.30.188
190.249.171.91
190.254.30.137
190.70.32.86
186.146.200.109
186.154.95.91
186.155.147.130
186.155.16.137
186.168.193.181
186.27.231.158
186.31.44.213
186.82.85.37
186.83.184.7
186.84.22.36
186.85.170.238
186.87.201.65
186.96.110.148
193.150.117.3
194.58.36.245
194.58.36.249
195.154.34.145
195.154.47.52
200.105.215.18
200.116.149.129
200.118.62.34
201.46.28.222
202.138.249.214
203.192.229.250
2

In [74]:
len(manually_filtered)

2396